In [1]:
#LIBRARIES
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from skimage.measure import label
from keras.applications.resnet50 import preprocess_input
import random
import pathlib

Using TensorFlow backend.


In [2]:
#MY DRIVE PATH

#path of where images that need to be converted (both train and test)
image_path = "3classes/HAM10000_train_by_class"
image_path_test = "3classes/HAM10000_test_by_class"

#path of where converted images are put. Please make this describe the nature of the conversion; i.e. train_dull_G-7-7-5-5_M-3
    #(the values of the parameters used to convert)
image_output_path = "3classes/HAM10000_train_dull_testing_file"
image_output_path_test = "3classes/HAM10000_test_dull_testing_file"

#turn on to create output directory (it does not currently exist). Make False if output directory exists. Note if False, will
#clear out the existing output directory; do not use if images you wish to be kept are already in output directory
create_output_directory = True

#This just triggers the reduction of the images down to imgnum; Set false to retain same amount of images as path you are
#converting from. Was put in due to the fact that utilities file can only upsample not downsample.
reducedataset = True
#change to desired number of images for training; this was simply to do downsampling as I was testing on 2k per class
imgnum = 2000

In [3]:
#HYPER-PARAMETERS FOR DULL RAZOR

#resize
resizeon = True #triggers resize
resizeW = 600 #height of resized image
resizeH = 450 #width of resized image
ResizeParam = [resizeW, resizeH]

#Resize alternative: Retain aspect ratio by cropping down to the central 'square'
resizecropon = True #triggers on, overrides resizeon
resizealtres = 450 #height and width of resulting square image

#dull razor
dullrazor = True #turn off to just blur
razorblur = "G" #M for median, G for Gaussian, have found Gaussian works better for this at cost of removal 
razorblurnum = 7 #blur number; higher values, greater blur, belive it has to be odd.
filterstructure = 7 #will form structure for kernel, higher values increases amount that will be removed. Modify with caution.
lowerbound = 5 #15 original; lower boundary for detecting hair in the grayscale image. Higher values will defintely remove hair 
               #but also may remove important features such as black spots on moles.
inpaintmat = 5 #radius of each point inpainted that is considered by the algorithm; higher numbers, more generalized end
               #colour that fills in the missing gaps created by removed hair. Ideally would prefer to keep low, but then may
               #fail to properly 

#blur
blur = True #turns on blur
normalblur = "M" #M for Median, G for Gaussian
blurnum = 3 #blur number, higher numbers = more blur, belive it has to be odd.

#attention mapping was removed due to poor results. If desired for the sake of completeness I can put it back but it may take
#some work.

In [4]:
#Creating relevant image paths
image_path_mel = os.path.join(image_path,"mel/")
image_path_bcc = os.path.join(image_path,'bcc/')
image_path_others = os.path.join(image_path,'others/')

image_path_mel_test = os.path.join(image_path_test,"mel/")
image_path_bcc_test = os.path.join(image_path_test,'bcc/')
image_path_others_test = os.path.join(image_path_test,'others/')

image_output_mel = os.path.join(image_output_path,'mel/')
image_output_bcc = os.path.join(image_output_path,'bcc/')
image_output_others = os.path.join(image_output_path,'others/')

image_output_mel_test = os.path.join(image_output_path_test,'mel/')
image_output_bcc_test = os.path.join(image_output_path_test,'bcc/')
image_output_others_test = os.path.join(image_output_path_test,'others/')

#for loading all images in the dataset
def loadImages(path):
    image_files = sorted([os.path.join(path, file)
                          for file in os.listdir(path)
                          if file.endswith('.jpg')])
    return image_files

#for actually converting them to images
def read_image_cv2(Dataset):  
  img = [cv2.imread(i, cv2.IMREAD_UNCHANGED,) for i in Dataset[:len(Dataset)]]
  return img

In [5]:
#only do this if you wish to reduce the dataset you are converting from down to imgnum as well. This is not recommended as this
#will likely mean you are deleting parts of your current base dataset, but is useful for comparison if you are testing against
#a smaller dataset set by imgnum; i.e. a 2k per class set, so the original also needs to be 2k per class set (as augmentation
#in utilities does not downsample is the number to be upsampled is below the current amount in others)
"""
files = os.listdir('3classes/HAM10000_train_by_class/others')
otherscount = 0
for path in pathlib.Path('3classes/HAM10000_train_by_class/others').iterdir():
    if path.is_file():
        otherscount += 1
for file in random.sample(files,otherscount-imgnum):
    os.remove(os.path.join('3classes/HAM10000_train_by_class/others',file))
"""

"\nfiles = os.listdir('3classes/HAM10000_train_by_class/others')\notherscount = 0\nfor path in pathlib.Path('3classes/HAM10000_train_by_class/others').iterdir():\n    if path.is_file():\n        otherscount += 1\nfor file in random.sample(files,otherscount-imgnum):\n    os.remove(os.path.join('3classes/HAM10000_train_by_class/others',file))\n"

In [6]:
#RESIZE IMAGE
def resize(Image,ResizeParam):
    #resize
    dim = (ResizeParam[0], ResizeParam[1])
    res_img = []
    if resizeon == True:
        for i in range(len(Image)):
            res = cv2.resize(Image[i], dim, interpolation=cv2.INTER_LINEAR)
            res_img.append(res)
    else:
        res_img = Image
    return res_img

#Resize but has to be down to square, assuming mole would be in the middle 'square'
def retainaspectresize(images,resolution):
    res_images = []
    for image in images:
        h,w,d = image.shape
        if h > w:
            croppedimg = image[0:w,int(round(h/2-w/2)):int(round(h/2+w/2))]
            resimg = cv2.resize(croppedimg, (resolution,resolution), interpolation=cv2.INTER_LINEAR)
        elif w > h:
            croppedimg = image[int(round(w/2-h/2)):int(round(w/2+h/2)),0:h]
            resimg = cv2.resize(croppedimg, (resolution,resolution), interpolation=cv2.INTER_LINEAR)
        else:
            resimg = cv2.resize(image,(resolution,resolution),interpolation=cv2.INTER_LINEAR)
        res_images.append(resimg)
    return res_images

#REMOVING HAIR USING DULL RAZOR
def dull_razor(ResizedImages):
    #dull razor
    hair_removed_images = []
    if dullrazor == True:
        for i in range(len(ResizedImages)):
            if razorblur == "M":
                tempimg = cv2.medianBlur(ResizedImages[i],razorblurnum)
            elif razorblur == "G":
                tempimg = cv2.GaussianBlur(ResizedImages[i], (razorblurnum, razorblurnum),0)
            else:
                tempimg = ResizedImages[i]
            gyimage = cv2.cvtColor(tempimg, cv2.COLOR_RGB2GRAY)
            filtersize = (filterstructure,filterstructure)
            kernelrazor = cv2.getStructuringElement(cv2.MORPH_RECT, filtersize)
            gyimage = cv2.morphologyEx(gyimage, cv2.MORPH_BLACKHAT, kernelrazor)

            retrazor, maskrazor = cv2.threshold(gyimage, lowerbound, 255, cv2.THRESH_BINARY)
            x = cv2.inpaint(ResizedImages[i], maskrazor, inpaintmat, cv2.INPAINT_TELEA)
            hair_removed_images.append(x)
        return hair_removed_images
    else:
        return ResizedImages

#BLUR AFTER REMOVING HAIR
def blur(HairRemovedImages):
    #blur
    if blur == True:
        if normalblur == "M":
            for i in range(len(HairRemovedImages)):
                HairRemovedImages[i] = cv2.medianBlur(HairRemovedImages[i], blurnum)
        elif normalblur == "G":
            for i in range(len(HairRemovedImages)): 
                HairRemovedImages[i] = cv2.GaussianBlur(HairRemovedImages[i], (blurnum, blurnum), 0)
    return HairRemovedImages

In [7]:
def delete_random_elems(input_list,n):
    to_delete = set(random.sample(range(len(input_list)),n))
    return[x for i,x in enumerate(input_list) if not i in to_delete]

In [9]:
#clear out output directory if create directory off, else creates directory
if create_output_directory == False:
    for root, dirs, files in os.walk(image_output_others):
        for file in files:
            os.remove(os.path.join(root, file))
    for root, dirs, files in os.walk(image_output_mel):
        for file in files:
            os.remove(os.path.join(root, file))
    for root, dirs, files in os.walk(image_output_bcc):
        for file in files:
            os.remove(os.path.join(root, file))
    for root, dirs, files in os.walk(image_output_others_test):
        for file in files:
            os.remove(os.path.join(root, file))
    for root, dirs, files in os.walk(image_output_mel_test):
        for file in files:
            os.remove(os.path.join(root, file))
    for root, dirs, files in os.walk(image_output_bcc_test):
        for file in files:
            os.remove(os.path.join(root, file))
elif create_output_directory == True:
    os.mkdir(image_output_path)
    os.mkdir(image_output_mel)
    os.mkdir(image_output_bcc)
    os.mkdir(image_output_others)
    os.mkdir(image_output_path_test)
    os.mkdir(image_output_mel_test)
    os.mkdir(image_output_bcc_test)
    os.mkdir(image_output_others_test)

In [10]:
import pathlib
#check number of files in those directories here
melcount = 0
for path in pathlib.Path(image_path_mel).iterdir():
    if path.is_file():
        melcount += 1
print("Mel Train: ",melcount)
otherscount = 0
for path in pathlib.Path(image_path_others).iterdir():
    if path.is_file():
        otherscount += 1
print("Others Train: ",otherscount)
bcccount = 0
for path in pathlib.Path(image_path_bcc).iterdir():
    if path.is_file():
        bcccount += 1
print("BCC Train: ",bcccount)
melcount = 0
for path in pathlib.Path(image_path_mel_test).iterdir():
    if path.is_file():
        melcount += 1
print("Mel Test: ",melcount)
otherscount = 0
for path in pathlib.Path(image_path_others_test).iterdir():
    if path.is_file():
        otherscount += 1
print("Others Test: ",otherscount)
bcccount = 0
for path in pathlib.Path(image_path_bcc_test).iterdir():
    if path.is_file():
        bcccount += 1
print("BCC Test: ",bcccount)

Mel Train:  2000
Others Train:  2000
BCC Train:  2000
Mel Test:  229
Others Test:  1710
BCC Test:  113


In [11]:
data = loadImages(image_path_others)
if reducedataset == True:
    data = delete_random_elems(data,len(data)-imgnum)
img = read_image_cv2(data)
if resizecropon == True:
    res_img = retainaspectresize(img,resizealtres)
else:
    res_img = resize(img, ResizeParam)
hair_removed_image = dull_razor(res_img)
dull_blur_images = blur(hair_removed_image)
imagecount = 0
for i in dull_blur_images:
    cv2.imwrite(os.path.join(image_output_others,str(imagecount)+".jpg"),i)
    imagecount=imagecount+1
    
data = loadImages(image_path_others_test)
img = read_image_cv2(data)
if resizecropon == True:
    res_img = retainaspectresize(img,resizealtres)
else:
    res_img = resize(img, ResizeParam)
hair_removed_image = dull_razor(res_img)
dull_blur_images = blur(hair_removed_image)
imagecount = 0
for i in dull_blur_images:
    cv2.imwrite(os.path.join(image_output_others_test,str(imagecount)+".jpg"),i)
    imagecount=imagecount+1
    
data = loadImages(image_path_mel)
if reducedataset == True:
    data = delete_random_elems(data,len(data)-imgnum)
img = read_image_cv2(data)
if resizecropon == True:
    res_img = retainaspectresize(img,resizealtres)
else:
    res_img = resize(img, ResizeParam)
hair_removed_image = dull_razor(res_img)
dull_blur_images = hair_removed_image_furtherbluring = blur(hair_removed_image)
imagecount = 0
for i in dull_blur_images:
    cv2.imwrite(os.path.join(image_output_mel,str(imagecount)+".jpg"),i)
    imagecount=imagecount+1
    
data = loadImages(image_path_mel_test)
img = read_image_cv2(data)
if resizecropon == True:
    res_img = retainaspectresize(img,resizealtres)
else:
    res_img = resize(img, ResizeParam)
hair_removed_image = dull_razor(res_img)
dull_blur_images = hair_removed_image_furtherbluring = blur(hair_removed_image)
imagecount = 0
for i in dull_blur_images:
    cv2.imwrite(os.path.join(image_output_mel_test,str(imagecount)+".jpg"),i)
    imagecount=imagecount+1
    
data = loadImages(image_path_bcc)
if reducedataset == True:
    data = delete_random_elems(data,len(data)-imgnum)
img = read_image_cv2(data)
if resizecropon == True:
    res_img = retainaspectresize(img,resizealtres)
else:
    res_img = resize(img, ResizeParam)
hair_removed_image = dull_razor(res_img)
dull_blur_images = hair_removed_image_furtherbluring = blur(hair_removed_image)
imagecount = 0
for i in dull_blur_images:
    cv2.imwrite(os.path.join(image_output_bcc,str(imagecount)+".jpg"),i)
    imagecount=imagecount+1

data = loadImages(image_path_bcc_test)
img = read_image_cv2(data)
if resizecropon == True:
    res_img = retainaspectresize(img,resizealtres)
else:
    res_img = resize(img, ResizeParam)
hair_removed_image = dull_razor(res_img)
dull_blur_images = hair_removed_image_furtherbluring = blur(hair_removed_image)
imagecount = 0
for i in dull_blur_images:
    cv2.imwrite(os.path.join(image_output_bcc_test,str(imagecount)+".jpg"),i)
    imagecount=imagecount+1

In [14]:
image_path_mel_test

'3classes/HAM10000_test_by_class/mel/'

In [13]:
import pathlib
#check number of files in those directories here
melcount = 0
for path in pathlib.Path(image_output_mel).iterdir():
    if path.is_file():
        melcount += 1
print("Mel Train: ",melcount)
otherscount = 0
for path in pathlib.Path(image_output_others).iterdir():
    if path.is_file():
        otherscount += 1
print("Others Train: ",otherscount)
bcccount = 0
for path in pathlib.Path(image_output_bcc).iterdir():
    if path.is_file():
        bcccount += 1
print("BCC Train: ",bcccount)
melcount = 0
for path in pathlib.Path(image_output_mel_test).iterdir():
    if path.is_file():
        melcount += 1
print("Mel Test: ",melcount)
otherscount = 0
for path in pathlib.Path(image_output_others_test).iterdir():
    if path.is_file():
        otherscount += 1
print("Others Test: ",otherscount)
bcccount = 0
for path in pathlib.Path(image_output_bcc_test).iterdir():
    if path.is_file():
        bcccount += 1
print("BCC Test: ",bcccount)

Mel Train:  2000
Others Train:  2000
BCC Train:  2000
Mel Test:  0
Others Test:  0
BCC Test:  0
